In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [3]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)
##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'

x = pd.read_csv(allplots_fc).groupby(['Latitude', 'Longitude']).first().reset_index()

fc_all = geemap.pandas_to_ee(x, latitude = "Latitude", longitude = "Longitude") ##FOR DAYMET AND GRIDMET

gce_clip = 'F:/Wetlands shapefiles/GCE domain/GCE_LTER_boundary.shp' ##Now, do the entire GA coast!
gce = geemap.shp_to_ee(gce_clip)

In [4]:
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

In [17]:
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.updateMask(image.select('SR_B3').mask()).rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.updateMask(image.select('SR_B3').mask()).rename('flats'))

def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.updateMask(image.select('SR_B4').mask()).rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.updateMask(image.select('SR_B4').mask()).rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.2)
    return image.updateMask(mask1)

In [18]:
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
         .filterBounds(fc_all) \
         .map(maskL8sr).map(addFLATSL5)

ls7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
         .filterBounds(fc_all) \
         .map(maskL8sr).map(addFLATSL7)

ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
         .filterBounds(fc_all) \
         .map(maskL8sr).map(addFLATSL8)

ls9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
         .filterBounds(fc_all) \
         .map(maskL8sr).map(addFLATSL9)

ultra_col = ls5.merge(ls7).merge(ls8).merge(ls9)

In [19]:
def classifyAndCount(img):
    flats = img.select('flats')
    
    flooded = flats.gte(0.2).rename('flooded')  # 1 or 0, already masked
    valid = flats.mask().rename('valid')        # 1 where valid, null otherwise
    
    return flooded.toFloat().addBands(valid)
    
binary = ultra_col.map(classifyAndCount)

flooded_sum = binary.select('flooded').sum()
valid_count = binary.select('valid').sum()

fraction_flooded = flooded_sum.divide(valid_count).rename('fraction_flooded')

In [20]:
results = fraction_flooded.addBands(valid_count).sampleRegions(
    collection=fc_all,
    scale=30,
    geometries=True
)

In [26]:
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['lightgray', 'yellow', 'red']
}

# Create a map and add the layer
m = geemap.Map(center=[37, -95], zoom=4)


m.addLayer(fraction_flooded, vis_params, 'Percent of flats > 0.2')

m.addLayer(fc_all, {}, 'sampling points')

m

Map(center=[37, -95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [25]:
def rasterExtraction(image):
    return image.sampleRegions(
        collection=fc_all,  # Use the subset of locations
        scale=30,
        tileScale=16  # Adjust tile scale for performance
    )

extracted_features = rasterExtraction(fraction_flooded)

fc_dict = extracted_features.getInfo()
features = [feature['properties'] for feature in fc_dict['features']]
df = pd.DataFrame(features)


# import os
# out_dir = 'C:/Users/arj26323/Downloads'
# filename = os.path.join(out_dir, "xxxcxx.csv")
# df.to_csv(filename, index=False)
